# Solving combinatorial optimization problems.
When solving an instance of a combinatorial optimization problem, one is generally interested in an object from a finite or, possibly, a countable infinite set that satisfies certain conditions or constraints. Such an object can be an integer, a set, a permutation, or a graph.
This notebook shows how to define and solve an instance of a knapsack problem.

## Knapsack.
The knapsack is another popular combinatorial problem in the scientific community. In a knapsack problem, one is given a set of items, each associated to a given value and size (such as the weight and/or volume), and a *knapsack* with a maximum capacity; solving an instance of a knapsack problem implies to *pack* a subset of items into the knapsack, so that the items’ total size does not exceed the knapsack’s capacity, and their total value is maximized. If the total size of the items exceeds the capacity, such a solution is considered unfeasible. There is a wide range of knapsack-based problems, many of which are NP-hard, and large instances of such problems can be approached only by using heuristic algorithms. In this release, the library contains two variants: the *0-1* and the *bounded* knapsack problems, implemented in the module ``knapsack`` as classes ``Knapsack01`` and ``KnapsackBounded``, respectively.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/1200px-Knapsack.svg.png" alt="Drawing" style="width: 400px;"/>

# 1. Create problems' instances.

Loads the necessary classes and functions.

In [40]:
# Imports PyTorch
import torch
# Imports problems
from gpol.problems.knapsack import KnapsackBounded
# Imports metaheuristics 
from gpol.algorithms.random_search import RandomSearch
from gpol.algorithms.local_search import HillClimbing
from gpol.algorithms.local_search import SimulatedAnnealing
from gpol.algorithms.genetic_algorithm import GeneticAlgorithm
# Imports operators
from gpol.operators.initializers import prm_rnd_vint, prm_rnd_mint
from gpol.operators.selectors import prm_tournament
from gpol.operators.variators import prm_rnd_int_ibound, one_point_xo

Creates an instance of ``KnapsackBounded``. The search space (*S*) of an instance of ``KnapsackBounded`` consists of the following key-value pairs:
- ``"capacity"`` as the maximum capacity of the *knapsack*;
- ``"n_dims"`` as the number of unique items in $S$;
- ``"weights"`` as the collection of items’ weights defined as a vector of type ``torch.float``;
-  ``"values"`` as the collection of items’ values defined as a vector of type ``torch.float``; and 
-  ``"bounds"`` as a $2$x$n$ tensor representing the minimum and the maximum number of copies allowed for each of the $n$ items in $S$.

In [41]:
# Defines the processing device and random state 's seed
device, seed =  "cpu", 0 # 'cuda' if torch.cuda.is_available() else 'cpu', 0
# Characterizes the problem: defines the maximum number of items and their maximum allowed repetitions
n_items, max_rep = 17, 4
# Randomly generates items ’ weights and values (for the sake of example)
torch.manual_seed (seed)  # sets the random state 
weights = torch.FloatTensor(n_items).uniform_(1, 9).to(device)
values = torch.FloatTensor(n_items).uniform_(0.5 , 20).to(device)
bounds = torch.stack((torch.ones(n_items), max_rep*torch.ones(n_items))).to(device)  # 1-4 copies per item
# Creates the search space
sspace = {"capacity": 160, "n_dims": n_items, "weights": weights, "values": values, "bounds": bounds}
# Creates an instance of KnapsackBounded
pi = KnapsackBounded(sspace=sspace, ffunction=torch.matmul, min_=False)

# 2. Choose and parametrize the algorithms.

## 2.1. Random search (RS).
The random search (RS) can be seen as thethe first rudimentary stochastic metaheuristic for problem-solving. Its strategy, far away from being *intelligent*, consists of randomly sampling $S$ for a given number of iterations. As such, the only search-parameter of an instance of ``RandomSearch`` is the initialization function (the ``initializer``). The function ``prm_rnd_vint`` returns a vector which follows a discrete uniform distribution between with user-specified lower and upper bounds.

The cell in below creates a dictionary called ``pars`` which stores algorithms' parameters. Each key-value pair stores the algorithm's type and a dictionary of respective search-parameters. The first key-value pair regards the RS.

In [42]:
# Defines a single-point (SP) initializer
sp_init = prm_rnd_vint(lb=0, ub=max_rep+1)  # + 1 because ~U{lb, ub}, with ub excluded 
# Defines RS's parameters
pars = {RandomSearch: {"initializer": sp_init}}

## 2.2. Hill climbing (HC).
The local search (LS) algorithms can be seen among the first intelligent search strategies that improve the functioning of the RS. They rely upon the concept of neighborhood which is explored at each iteration by sampling from $S$ a limited number of neighbors of the best-so-far solution. Usually, the LS algorithms are divided in two branches. In the first branch, called hill climbing (HC), or hill descent for the minimization problems, the best-so-far solution is replaced by its neighbor when the latter is at least as good as the former.

The cell in below adds ``HillClimbing`` to ``pars``. Note that, unlike it was for ``RandomSearch``, an instance of ``HillClimbing`` also requires the specification of a neighbor-generation function (``"nh_function"``) and the neighborhood's size (``"nh_size"``). In this example, a randomly selected index is replaced in the candidate solution, with a probability of $0.3$; the replacement value is a randomly drawn integer in the range [0, ``max_rep``]. Note that the very same initialization function is used for both ``RandomSearch`` and ``HillClimbing``. 

In [43]:
# Defines the size of the population/neighborhood 
nh_size = 100
# Defines neighbor-generation function with the respective parameters
nh_function = prm_rnd_int_ibound(prob=0.3, lb=0, ub=max_rep)
# Defines HC's parameters
pars[HillClimbing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size}

## 2.3. Simulated annealing (SA).
The second branch, called simulated annealing (SA), extends HC by formulating a non-negative probability of replacing the best-so-far solution by its neighbor when the latter is worse. Traditionally, such a probability is small and decreases as the search advances. The strategy adopted by SA is especially useful when the search is prematurely tagnated at a locally sub-optimal point in $S$.

The cell in below adds ``SimulatedAnnealing`` to ``pars``. 

In [44]:
# Defines SA's parameters
pars[SimulatedAnnealing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size, "control": 1.0, "update_rate": 0.9}

## 2.4. Genetic algorithm (GA).
Based on the number of candidate solutions they handle at each step, the metaheuristics can be categorized into single-point (SP) and population-based (PB) approaches. 
The search procedure in the SP metaheuristics is generally guided by the information provided by a single candidate solution from $S$, usually the best-so-far solution, that is gradually evolved in a well-defined manner in hope to find the global optimum. The abovementioned HC and SA are examples of SP metaheuristics as the search is performed by exploring the neighborhood $N(i)$, where $i$ is the current best solution. Contrarily, the search procedure in PB metaheuristics is generally guided by the information shared by a set of candidate solutions and the exploitation of its collective behavior in different ways. In abstract terms, one can say that every PB metaheuristics shares, at least, the following two features: an object representing the set of simultaneously exploited candidate solutions (i.e., the population), and a procedure to *move* them across $S$.

Genetic Algorithm (GAs) is a meta-heuristic introduced by J. Holland which was strongly inspired by Darwin's theory of evolution by means of natural selection. Conceptually, the algorithm starts with a random-like population of candidate solutions (called *chromosomes*). Then, by mimicking the natural selection and genetically inspired variation operators, such as the crossover and the mutation, the algorithm breeds a population of the next-generation candidate solutions (called the *offspring population*), that replaces the previous population (a.k.a. the *parent population*). This procedure is iterated until reaching some stopping criteria, like a maximum
number of iterations (also called *generations*).

The cell in below adds ``GeneticAlgorithm`` to ``pars``. It uses a slightly different initializer, specially designed to efficiently initialize PB metaheuristics, called ``prm_rnd_mint``, that returns a matrix generated under discrete uniform distribution between with user-specified lower and upper bounds. Note that the mutation function is the neighbor-generation function, that was used for the aforementioned LS algorithms, and the population's size is equivalent to the neighborhood's size; this is done to foster the equivalency between LS and PB metaheuristics. Finally, the crossover is one-point, the elite is always a member of the population and parents' reproduction is enabled (``elitism=True`` and ``reproduction=True``).

In [45]:
# Defines a population-based (PB) initializer
pb_init = prm_rnd_mint(0, max_rep+1)
# Defines GA's parameters
pars[GeneticAlgorithm] = {"pop_size": nh_size, "initializer": pb_init, "selector": prm_tournament(pressure=0.05), "mutator": nh_function, 
                          "crossover": one_point_xo, "p_m": 0.3, "p_c": 0.7, "elitism": True, "reproduction": True}

# 3. Executes the experiment.

Note that *many* parameters and functions are shared across different algorithms in the experiment. This allows to increase the control and comparability between different algorithmic approaches when solving a given problem's instance.

In [46]:
for isa_type, isa_pars in pars.items():
    print(isa_type)
    for p_name, p_val in isa_pars.items():
        print("\t", p_name, p_val)

<class 'gpol.algorithms.random_search.RandomSearch'>
	 initializer <function prm_rnd_vint.<locals>.rnd_vint at 0x000001324FFB9CA0>
<class 'gpol.algorithms.local_search.HillClimbing'>
	 initializer <function prm_rnd_vint.<locals>.rnd_vint at 0x000001324FFB9CA0>
	 nh_function <function prm_rnd_int_ibound.<locals>.rnd_int_ibound at 0x000001324FF9CEE0>
	 nh_size 100
<class 'gpol.algorithms.local_search.SimulatedAnnealing'>
	 initializer <function prm_rnd_vint.<locals>.rnd_vint at 0x000001324FFB9CA0>
	 nh_function <function prm_rnd_int_ibound.<locals>.rnd_int_ibound at 0x000001324FF9CEE0>
	 nh_size 100
	 control 1.0
	 update_rate 0.9
<class 'gpol.algorithms.genetic_algorithm.GeneticAlgorithm'>
	 pop_size 100
	 initializer <function prm_rnd_mint.<locals>.rnd_mint at 0x000001324FFB93A0>
	 selector <function prm_tournament.<locals>.tournament at 0x000001324FFB9820>
	 mutator <function prm_rnd_int_ibound.<locals>.rnd_int_ibound at 0x000001324FF9CEE0>
	 crossover <function one_point_xo at 0x0000

Defines the computational resources for the experiment: the number of iterations.

In [47]:
n_iter = 30

Loops the afore-defined ``pars`` dictionary containing algorithms' and the underlying parameters. Note that besides algorithm-specific parameters, the constructor of an instance of a search algorithm also receives the random state to initialize a pseudorandom number generator (called ``seed``), and the specification of the processing ``device`` (either CPU or GPU).

The ``solve`` method has the same signature for all the search algorithms and, in this example, includes the following parameters: 
-  ``n_iter``: number of iterations to conduct the search;
-  ``tol``: minimum required fitness improvement for ``n_iter_tol`` consecutive iterations to continue the search. When the fitness is not improving by at least ``tol`` for ``n_iter_tol`` consecutive iterations, the search will be automatically interrupted;
-  ``n_iter_tol``: maximum number of iterations to not meet ``tol`` improvement;
-  ``verbose``: verbosity's detail-level;
-  ``log``: log-files' detail-level (if exists).

In [48]:
for isa_type, isa_pars in pars.items():
    isa = isa_type(pi=pi, **isa_pars, seed=seed, device=device)
    # n_iter*pop_size if isinstance(isa, RandomSearch) else n_iter  # equivalency for the RS
    isa.solve(n_iter=n_iter, tol=10, n_iter_tol=5, verbose=2, log=0)
    print("Algorithm: {}".format(isa_type.__name__))
    print("Best solution's fitness: {:.3f}".format(isa.best_sol.fit))
    print("Best solution:", isa.best_sol.repr_, end="\n\n")

-------------------------------------------------
           |           Best solution            |
-------------------------------------------------
Generation   Length   Fitness              Timing
0            33       294.668               0.007
1            33       294.668               0.000
2            33       294.668               0.000
3            33       294.668               0.000
4            33       294.668               0.000
5            33       294.668               0.000
Algorithm: RandomSearch
Best solution's fitness: 294.668
Best solution: tensor([2., 1., 1., 2., 1., 3., 1., 3., 1., 1., 4., 3., 4., 2., 2., 1., 1.])

--------------------------------------------------------------------------------------
           |           Best solution              |           Neighborhood           |
--------------------------------------------------------------------------------------
Generation | Length   Fitness              Timing | AVG Fitness           STD Fitness
0  